In [1]:
import numpy
from urllib.request import urlopen
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
import gzip
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import math
import numpy as np
import random

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
def readCSV(path):
    f=gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

In [4]:
path='./train_Category.json.gz'
reviews=[]
for l in readGz(path):
    reviews.append(l)
    

In [5]:
review=reviews[:10000]

In [6]:
review[0]

{'n_votes': 0,
 'review_id': 'r99763621',
 'user_id': 'u17334941',
 'review_text': "Genuinely enthralling. If Collins or Bernard did invent this out of whole cloth, they deserve a medal for imagination. Lets leave the veracity aside for a moment - always a touchy subject when it comes to real life stories of the occult - and talk about the contents. \n The Black Alchemist covers a period of two years in which Collins, a magician, and Bernard, a psychic, undertook a series of psychic quests that put them in opposition with the titular Black Alchemist. As entertainment goes, the combination of harrowing discoveries, ancient lore, and going down the pub for a cigarette and a Guinness, trying to make sense of it all while a hen party screams at each other, is a winner. It is simultaneously down to earth and out of this world. \n It reads fast, both because of the curiousity and because Collins has a very clear writing style. Sometimes its a little clunky or over repetitive and there's a fe

In [7]:
punctuation = set(string.punctuation)
bigramDic=defaultdict(int)
unigramDic=defaultdict(int)
for r in review:
    r=''.join([i.lower() for i in list(r['review_text']) if i not in punctuation]).split()
    for n in range(len(r)-1):
        bigram=str(r[n]+' '+r[n+1])
        unigramDic[r[n]]+=1
        bigramDic[bigram]+=1
    unigramDic[r[len(r)-1]]+=1
    
    
    

### question 1

In [8]:
counts=[(bigramDic[x], x) for x in bigramDic]
counts.sort(reverse=True)

In [9]:
counts[:5]

[(7927, 'of the'),
 (5850, 'this book'),
 (5627, 'in the'),
 (3189, 'and the'),
 (3183, 'is a')]

### question 2

In [10]:
###festures: 1000 most common bigrams
###label: rating
###MSE: regularized regression 1.0

In [11]:
num=1000
count=counts[:num]

In [12]:
bigram_index=dict(zip([x[1] for x in count],range(1000)))
bigrams=[x[1] for x in count]

In [13]:
def feature(r):
    feat=[0]*1000
    r_ls=''.join([i.lower() for i in list(r['review_text']) if i not in punctuation]).split()
    for n in range(len(r_ls)-1):
        bi_str=str(r_ls[n]+" "+r_ls[n+1])
        if bi_str in bigrams:
            feat[bigram_index[bi_str]]+=1
    feat.append(1)
    return feat

In [14]:
features=[]
for r in review:
    feat=feature(r)
    features.append(feat)

In [15]:
labels=[r['rating'] for r in review]

In [16]:
#regularized regression
from sklearn import linear_model
clf=linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(features, labels)


Ridge(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [17]:
predictions=clf.predict(features)

In [18]:
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(labels, predictions)
MSE

1.0178804824879226

### MSE if 1.01788

### question 3

In [19]:
#1000 features will be some combination of unigrams and bigrams

In [20]:
bi_counts=[(bigramDic[x],x) for x in bigramDic]
uni_counts=[(unigramDic[x],x) for x in unigramDic]

In [21]:
all_counts=bi_counts+uni_counts
all_counts.sort(reverse=True)
all_count=all_counts[:1000]

In [22]:
all_count_index=dict(zip([x[1] for x in all_count],range(len(all_count))))
all_count_words=list(all_count_index.keys())


In [23]:
def feature2(r):
    feat=[0]*1000
    r_ls=''.join([i.lower() for i in list(r['review_text']) if i not in punctuation]).split()
    for n in range(len(r_ls)-1):
        bigram=str(r_ls[n]+' '+r_ls[n+1])
        unigram=r_ls[n]
        if bigram in all_count_words:
            feat[all_count_index[bigram]]+=1
        if unigram in all_count_words:
            feat[all_count_index[unigram]]+=1
    if r_ls[-1] in all_count_words:
        feat[all_count_index[r_ls[-1]]]+=1
    feat.append(1)
    return feat
    

In [24]:
features2=[]
for r in review:
    feat=feature2(r)
    features2.append(feat)

In [58]:
clf.fit(features2, labels)
prediction2=clf.predict(features2)

In [59]:
MSE2=mean_squared_error(labels, prediction2)
MSE2

0.9683580241694552

### the MSE is 0.968

### question 4

In [27]:
#inverse document frequency of the words ‘stories’, ‘magician’, ‘psychic’, ‘writing’, and ‘won- der’
#What are their tf-idf scores in the first review

In [29]:
import math
import numpy as np
idf_dic=defaultdict(float)
for r in review:
    r_ls=''.join([i.lower() for i in list(r['review_text']) if i not in punctuation]).split()
    list_words=['stories','magician', 'psychic', 'writing', 'wonder']
    for w in r_ls:
        if w in list_words:
            idf_dic[w]+=1
            list_words.remove(w)  
for w in list_words:
    idf_dic[w]=np.log10(len(review)/idf_dic[w])


### idf score for ['stories','magician', 'psychic', 'writing', 'wonder'] in dataset

In [30]:
example=review[0]
list_words=['stories','magician', 'psychic', 'writing', 'wonder']
ls=''.join([i.lower() for i in list(example['review_text']) if i not in punctuation]).split()
tf=[]
for i in list_words:
    ct=ls.count(i)
    tf.append(ct)

In [31]:
tfidf_ls=np.array(tf)*np.array(list(idf_dic.values()))
tfidf_ls

array([1.11747546, 2.65757732, 5.20411998, 0.99783394, 1.76700389])

### tfidf score for the first review

### question 5

In [32]:
#tfidf scores for the 1000 most common unigrams as features
#MSE
uni_counts=[(unigramDic[x],x) for x in unigramDic]
uni_counts.sort(reverse=True)
count=uni_counts[0:1000]
count[:3]

[(73431, 'the'), (44301, 'and'), (39577, 'a')]

In [33]:
wordID=dict(zip([x[1] for x in count], range(len(count))))
word=[x[1] for x in count]

In [34]:
#idf for word
idf_dic=defaultdict(float)
for r in review:
    r_ls=''.join([i.lower() for i in list(r['review_text']) if i not in punctuation]).split()
    words=[x[1] for x in count]
    for w in r_ls:
        if w in words:
            idf_dic[w]+=1
            words.remove(w)  

In [35]:
for w in word:
    
    idf_dic[w]=np.log10(len(review)/(idf_dic[w]+1))
idf={k: idf_dic[k] for k in word}


In [36]:
def feat3(r):
    feat=[0]*1000
    tf_dic=defaultdict(int)
    r_ls=''.join([i.lower() for i in list(r['review_text']) if i not in punctuation]).split()
    for i in r_ls:
        if i in word:
            tf_dic[i]+=1
    tf={k: tf_dic[k] for k in word}
    for i in range(1000):
        feat[i]=tf[word[i]]*idf[word[i]]
    feat.append(1)
    return feat

In [61]:
feature3=[]
for r in review:
    feat=feat3(r)
    feature3.append(feat)

In [62]:
clf.fit(feature3, labels)
pred3=clf.predict(feature3)

In [63]:
MSE3=mean_squared_error(labels, pred3)
MSE3

0.9660142273530128

### mse is 0.966


### question 6

In [40]:
#which other review has the highest cosine similarity compared to the first review, in terms of their 
#tf-idf representations (considering unigrams only)

In [41]:
r1=np.array(feature3[0])
r=np.array(feature3[1:])
def cs(r1,r2):
    numer=np.dot(r1, r2)
    denum=(np.dot(r1,r1)*np.dot(r2,r2))**(1/2)
    return numer/denum
    

In [1]:
cs_ls=[]
for i in range(len(r)):
    cos=cs(r1,r[i])
    cs_ls.append(cos)
index=cs_ls.index(max(cs_ls))+1

NameError: name 'r' is not defined

In [43]:
print(review[index]['review_id'])
print(review[index]['review_text'])

r81495268
This review is going to be very personal. I will probably not talk about the writing, the plot or the characters the way that I normally would. Dear Martin is about 17-year-old Justyce McAllistera a highly successful black student who attends a primarily all-white private school. One day, he is helping a drunk friend and arrested. In response, he starts writing letters to Dr, Martin Luther King Jr. as a way to cope with his newly sharpened awareness of the world around him 
 The world needed this book right now. This is a book that every school needs. I am going to be buying copies for friends and family. I am a teacher. I can see how this book would be an excellent way to help students start having these difficult conversations. Because until we teach young people how to recognize and talk about racism, sexism, homophobia, and xenophobia and whatever else needs to get talked about nothing will change. They will unconsciously internalize the system's biases. They won't see th

### question 7

In [44]:
import random

In [45]:
# suffle the data
index=random.sample(range(len(reviews)),30000)
train_index=index[:10000]
valid_index=index[10000:20000]
test_index=index[20000:]

In [46]:
# get the train, validation and test data
train=[]
for i in train_index:
    train.append(reviews[i])
valid=[]
for i in valid_index:
    valid.append(reviews[i])
test=[]
for i in test_index:
    test.append(reviews[i])

In [47]:
data=[train, valid, test]
y=[]
y_v=[]
y_t=[]
Y=[y, y_v, y_t]
train_text=[]
valid_text=[]
test_text=[]
raw_text=[train_text, valid_text, test_text]
for index in range(len(data)):
    for r in data[index]:
        rate=r['rating']
        Y[index].append(rate)
        r=list(''.join([i.lower() for i in list(r['review_text'])]))
        for n in range(len(r)):
            if r[n] in punctuation:
                r[n]=str(' '+r[n]+' ')
        r=''.join(r).split()
        raw_text[index].append(r)


In [48]:
def uni(train_text, valid_text, test_text, ifCount, ifRemove):
    train_features=[]
    val_features=[]
    test_features=[]
    features_ls=[train_features, val_features, test_features]
    text_ls=[train_text, valid_text, test_text]
    if ifRemove:
        for text in text_ls:
            for n in range(len(text)):
                text[n]=[i for i in text[n] if i not in punctuation]
                
     
    #get the topwords of training data for unigram 
    train_dic=defaultdict(int)
    for r in train_text:
        for word in r:
            train_dic[word]+=1
    wordCount=[(train_dic[x],x) for x in train_dic]
    wordCount.sort(reverse=True)
    topwords=[x[1] for x in wordCount[:1000]]
    topwordsID=dict(zip(topwords, range(1000)))
       
    
    for i in range(3):
        for r in text_ls[i]:
            feat=[0]*1000
            for word in r:
                if word in topwords:
                    feat[topwordsID[word]]+=1
            features_ls[i].append(feat)


         
    if ifCount==False: #tdidf=tf*idf
        #get the idf for topwords of training data for unigram
        idf_uni=[0]*1000
        for r in train_text:
            words=topwords
            for w in list(set([i for i in r if i in words])):
                idf_uni[topwordsID[w]]+=1
        idf_uni=np.log10(len(train_text)/np.array(idf_uni))
         
        for i in range(3):
            feat_ls=features_ls[i]
            for n in range(len(feat_ls)):
                feat_ls[n]=feat_ls[n]*idf_uni
    return features_ls
        
       
        
        
    


In [49]:
def bi(train_text, valid_text, test_text, ifCount, ifRemove):
    train_features=[]
    val_features=[]
    test_features=[]
    features_ls=[train_features, val_features, test_features]
    text_ls=[train_text, valid_text, test_text]
    
    if ifRemove:
        for text in text_ls:
            for n in range(len(text)):
                text[n]=[i for i in text[n] if i not in punctuation]
    
    #convert train, valid, test data to bigram
    for i in range(3):
        for r in text_ls[i]:
            r_ls=[]
            index=text_ls[i].index(r)
            for n in range(len(r)-1):
                string=str(r[n]+' '+r[n+1])
                r_ls.append(string)
            text_ls[i][index]=r_ls
    
    
     
    #get the topwords of training data for unigram 
    train_dic=defaultdict(int)
    for r in train_text:
        for string in r:
            train_dic[string]+=1
    wordCount=[(train_dic[x],x) for x in train_dic]
    wordCount.sort(reverse=True)
    topwords=[x[1] for x in wordCount[:1000]]
    topwordsID=dict(zip(topwords, range(1000))) 

        
    #count
    for i in range(3):
        for r in text_ls[i]:
            feat=[0]*1000
            for word in r:
                if word in topwords:
                    feat[topwordsID[word]]+=1
            features_ls[i].append(feat)


         
    if ifCount==False: #tdidf=tf*idf
        #get the idf for topwords of training data for unigram
        idf_bi=[0]*1000
        for r in train_text:
            words=topwords
            for w in list(set([i for i in r if i in words])):
                idf_bi[topwordsID[w]]+=1
        idf_bi=np.log10(len(train_text)/np.array(idf_bi))
         
        for i in range(3):
            feat_ls=features_ls[i]
            for n in range(len(feat_ls)):
                feat_ls[n]=feat_ls[n]*idf_bi
    return features_ls


In [50]:
X_tr=[]
X_v=[]
X_t=[]
####uni
tr,v,t=train_text, valid_text, test_text
#uni remove count
ls=uni(tr, v, t, True, True)
X_tr.append(ls[0])
X_v.append(ls[1])
X_t.append(ls[2])

tr,v,t=train_text, valid_text, test_text
#uni remove tfidf
ls=uni(tr, v, t, False, True)
X_tr.append(ls[0])
X_v.append(ls[1])
X_t.append(ls[2])

tr,v,t=train_text, valid_text, test_text
#uni preserve count
ls=uni(tr, v, t, True, False)
X_tr.append(ls[0])
X_v.append(ls[1])
X_t.append(ls[2])

tr,v,t=train_text, valid_text, test_text
#uni preserve tfidf
ls=uni(tr, v, t, False, False)
X_tr.append(ls[0])
X_v.append(ls[1])
X_t.append(ls[2])

####bi
tr,v,t=train_text, valid_text, test_text
#bi remove count
ls=bi(tr, v, t, True, True)
X_tr.append(ls[0])
X_v.append(ls[1])
X_t.append(ls[2])

tr,v,t=train_text, valid_text, test_text
#bi remove tfidf
ls=bi(tr, v, t, False, True)
X_tr.append(ls[0])
X_v.append(ls[1])
X_t.append(ls[2])

tr,v,t=train_text, valid_text, test_text
#bi preserve count
ls=bi(tr, v, t, True, False)
X_tr.append(ls[0])
X_v.append(ls[1])
X_t.append(ls[2])

tr,v,t=train_text, valid_text, test_text
#bi preserve tfidf
ls=bi(tr, v, t, False, False)
X_tr.append(ls[0])
X_v.append(ls[1])
X_t.append(ls[2])




In [51]:
result=[]
for i in range(8):
    MSE=[]
    parameters=[0.01, 0.1, 1, 10, 100]
    for c in parameters:
        model=linear_model.Ridge(c, fit_intercept=False)
        model.fit(X_tr[i], y)
        prediction=model.predict(X_v[i])
        mse_v=mean_squared_error(y_v, prediction)
        MSE.append(mse_v)
    bestC=parameters[MSE.index(max(MSE))]
    clf=linear_model.Ridge(bestC, fit_intercept=False)
    clf.fit(X_tr[i],y)
    pred=clf.predict(X_t[i])
    mse=mean_squared_error(y_t, pred)
    result.append(mse)





In [52]:
result

[7.503796908591908,
 7.5038681552506645,
 7.503796908591908,
 7.5038681552506645,
 8.104043962118938,
 8.913924376101305,
 10.207440563059665,
 11.81079622343473]

In [53]:
gram=["uni", "bi"]
remove=["remove", "preserve"]
count=["count","tfidf"]
strings=[]
for g in gram:
    for r in remove:
        for c in count:
            string=str(g+' '+r+' '+c)
            strings.append(string)

            
dict(zip(strings, result))      

{'uni remove count': 7.503796908591908,
 'uni remove tfidf': 7.5038681552506645,
 'uni preserve count': 7.503796908591908,
 'uni preserve tfidf': 7.5038681552506645,
 'bi remove count': 8.104043962118938,
 'bi remove tfidf': 8.913924376101305,
 'bi preserve count': 10.207440563059665,
 'bi preserve tfidf': 11.81079622343473}

In [54]:
import pandas as pd
pd.DataFrame(dict(zip(strings, result)), index=range(1))

,uni remove count,uni remove tfidf,uni preserve count,uni preserve tfidf,bi remove count,bi remove tfidf,bi preserve count,bi preserve tfidf
0,7.503797,7.503868,7.503797,7.503868,8.104044,8.913924,10.207441,11.810796
